<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/nn_7_ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7. Autoencoders for image generation and novelty detection

In [ ]:
!pip install tensorflow-gpu==1.15.3
!pip install livelossplot
!pip install keras==2.2.5
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras import Input, Model
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, UpSampling2D, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist, cifar10
from keras.utils import to_categorical
from livelossplot import PlotLossesKeras
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)
!nvidia-smi -L

In [ ]:
# set random seeds for more reproducible results
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(43)

In [ ]:
# load dataset
(raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = mnist.load_data()
#(raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = cifar10.load_data()
raw_x_train = np.expand_dims(raw_x_train, axis=-1)
raw_x_test = np.expand_dims(raw_x_test, axis=-1)
print(raw_x_train.shape, raw_y_train.shape)
train_size = len(raw_y_train)
test_size = len(raw_y_test)
xdim = raw_x_train.shape[1]
ydim = raw_x_train.shape[2]
print(raw_x_train.dtype, raw_y_train.dtype)

In [ ]:
plt.imshow(raw_x_train[0,...,0], cmap='gray')

In [ ]:
print(np.min(raw_x_train), np.max(raw_x_train), np.median(raw_x_train))
print(np.unique(raw_y_train, return_counts=True))
print(np.unique(raw_y_test, return_counts=True))

In [ ]:
x_train, y_train = shuffle(raw_x_train, raw_y_train, random_state=44)
x_train = x_train/255
x_test = raw_x_test/255
y_test = raw_y_test

x_train = np.asarray([x for x,y in zip(x_train,y_train) if y in (2,3,5,6,8)])
n_classes = len(np.unique(y_train))

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

#helper function to plot in grids
def show_grid(image_list,nrows,ncols,label_list=None,show_labels=False,savename=None,figsize=(10,10),showaxis='off'):
    if type(image_list) is not list:
        if(image_list.shape[-1]==1):
            image_list = [image_list[i,:,:,0] for i in range(image_list.shape[0])]
        elif(image_list.shape[-1]==3):
            image_list = [image_list[i,:,:,:] for i in range(image_list.shape[0])]
    fig = plt.figure(None, figsize,frameon=False)
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(nrows, ncols),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     share_all=True,
                     )
    for i in range(nrows*ncols):
        ax = grid[i]
        ax.imshow(image_list[i],cmap='Greys_r')  # The AxesGrid object work as a list of axes.
        ax.axis('off')
        if show_labels:
            ax.set_title(label_list[i])
    if savename != None:
        plt.savefig(savename,bbox_inches='tight')

show_grid(raw_x_train[:32],4,8,label_list=np.squeeze(raw_y_train[:32]),show_labels=True,figsize=(20,10))

In [ ]:
def get_model():
    inputs = Input(shape=(xdim, ydim,1))
    f = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(inputs)
    f = MaxPooling2D()(f)
    f = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(f)
    f = MaxPooling2D()(f)
    f = Conv2D(filters=2, kernel_size=3, activation='relu', padding='same')(f)
    encoded = MaxPooling2D(padding='same')(f)
    
    f = Conv2D(filters=2, kernel_size=3, activation='relu', padding='same')(encoded)
    f = UpSampling2D()(f)
    f = Conv2D(filters=8, kernel_size=3, activation='relu', padding='same')(f)
    f = UpSampling2D()(f)
    f = Conv2D(filters=8, kernel_size=3, activation='relu')(f)
    f = UpSampling2D()(f)
    outputs = Conv2D(filters=1, kernel_size=3, activation='sigmoid', padding='same')(f)
    
    return Model(inputs=inputs, outputs=outputs), Model(inputs=inputs, outputs=encoded)

In [ ]:
model, encoder = get_model()

print(model.summary())
model.compile(optimizer='adadelta', loss='binary_crossentropy')
history = model.fit(x=x_train, y=x_train, batch_size=512, epochs=300, validation_split=0.2, callbacks=[PlotLossesKeras()])

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
x_normal = np.asarray([x for x,y in zip(x_test,y_test) if y in (2,3,5,6,8)])
x_novel = np.asarray([x for x,y in zip(x_test,y_test) if y==4])
recon_normal = model.predict(x_normal)
recon_novel = model.predict(x_novel)

In [ ]:
plt.imshow(x_normal[0,...,0], cmap='gray')


In [ ]:
plt.imshow(recon_normal[0,...,0],cmap='gray')

In [ ]:
plt.imshow(x_novel[0,...,0], cmap='gray')

In [ ]:
plt.imshow(recon_novel[0,...,0],cmap='gray')

In [ ]:
error_normal = -np.mean(np.log(recon_normal+1e-19)*x_normal+np.log(1-recon_normal+1e-19)*(1-x_normal),axis=(1,2,3))
error_novel = -np.mean(np.log(recon_novel+1e-19)*x_novel+np.log(1-recon_novel+1e-19)*(1-x_novel),axis=(1,2,3))
print(np.mean(error_normal),np.mean(error_novel))
plt.hist(error_normal, 20, density=True, alpha=0.5, label='normal')
plt.hist(error_novel, 20, density=True, alpha=0.5, label='novel')

In [ ]:
code_normal = np.reshape(encoder.predict(x_normal),(-1,32))
code_novel = np.reshape(encoder.predict(x_novel),(-1,32))

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca')
low = tsne.fit_transform(np.vstack((code_normal,code_novel)))

In [ ]:
t_normal = low[:len(code_normal)]
t_novel = low[len(code_normal):]
fig=plt.figure(figsize=(9,6), dpi= 80, facecolor='w', edgecolor='k')
plt.scatter(t_normal[:,0],t_normal[:,1],color='b',s=2)
plt.scatter(t_novel[:,0],t_novel[:,1],color='r',alpha=0.5,s=2)
plt.show()